## c) Carga selectiva de grupos de enfermedades


Como observamos antes en la sección sobre la estructura de la columna "Causa de muerte", tenemos causas agrupadas, y en concreto las que queremos seleccionar vienen marcadas por las siguientes características:

⦁	Empieza por tres dígitos, seguidos por un guion y otros tres dígitos.

⦁	No totaliza todas las causas de fallecimiento.

La función de selección a utilizar será la elaborada en el apartado anterior y por lo tanto no se profundizará en explicaciones acerca de ella, ya que las podemos encontrar en el notebook correspondiente al apartado b.

Como ya se citó anteriormente, algunas causas de muerte, como "082  XVI.Afecciones originadas en el periodo perinatal" son dejadas de momento fuera por la siguiente función de selección, aunque en apartados posteriores la necesitaremos por tanto le haremos alguna modificación para que la incluya.

Aparte de cargar las filas deseadas, queremos cambiar el formato en el que se muestran para que las columnas representadas sean las siguientes:

    - Código numérico, que incluye los índices en arabescos y el numeral romano.
    - Numeral romano correspondiente al código numérico anterior.
    - Numeral árabe correspondiente a la traducción del numeral romano previo.
    - Nombre del grupo, sin códigos numéricos de ningún tipo, tan solo el nombre.
    - Año correspondiente al periodo temporal de los datos de esa fila.
    - Total de fallecimientos durante ese periodo debido a ese grupo de causas.
    
Con todo ello en mente, las funciones previamente definidas que vamos a utilizar son, la que reconoce si la causa de muerte de una de las filas se corresponde con una de las agrupadas, definida en el apartado b.2, un traductor de numerales romanos a árabes enteros, que se definió en el apartado b.1, una función que nos permite hacer determinadas divisiones de los strings de la columna "Causa de muerte" para poder separarlos en las columnas deseadas.

Vamos allá.

Cargamos nuestra función de filtro, definida en el apartado b.2:

In [1]:
import re

def es_grupo_y_no_total(causa):
    
    """
    Función que comprueba si un determinado string cumple las siguientes condiciones:
    
       - Empieza por tres dígitos, seguidos por un guion y otros tres dígitos.
       - No totaliza **todas** las causas de fallecimiento.
    
    Que quedan sintetizadas en las varaibles "patron" y "patron2" definidas durante la función.
        
    Parameters
    ----------
    causa : string
        String con la causa de muerte, a analizar para ver si cumple las condiciones.
    
    Returns
    -------
    Un valor boolean indicando si cumple las condiciones (True) o no (False).
    
    
    Example
    -------
    >>> es_grupo_y_no_total("001-008 I.Enfermedades infecciosas y parasitarias")
    True
    >>> es_grupo_y_no_total("001-102 I-XXII.Todas las causas Mujeres De 10 a 14 años 1983 308")
    False
    >>> es_grupo_y_no_total("XV.Embarazo")
    False
    """
    
    # Definimos nuestras expresiones a reconocer, la primera es una expresión regular que quiere decir:
    # 3 dígitos, seguidos por un guión y otros 3 dígitos.

    patron = "\d{3}-\d{3}"
    patron2 = "Todas las causas"

    # Lanzamos un bucle que busca, en primer lugar, si el string comienza con el patrón: "\d{3}-\d{3}".
    # En caso de que lo cumpla, comprueba si el string contiene el string prohibido: "Todas las causas".
    
    if re.match(patron, causa) != None:
        
        if re.search(patron2, causa) != None:
            return(False)
    
        else:
            return(True)

    else:
        return(False)

Cargamos nuestra función de traducción de numerales romanos, definida en el apartado b.1:

In [2]:
def romano_a_entero(num):
    
    """
    Función que transforma string interpretados como números romanos a números enteros desde el 1 hasta el 1000.
        
    Parameters
    ----------
    num : string
        String que representa un número romano.
    
    Returns
    -------
    valor: int
        Equivalente entero al número romano.
    string
        Si el número introducido no es un numeral romano nos lo indicará.
    
    
    Example
    -------
    >>> romano_a_entero("IV")
    4
    >>> romano_a_entero("CXXIX")
    129
    """

    # Definimos un diccionario con los dígitos romanos. Observar que como tienen una lógica aditiva y sustractiva, 
    # es más útil definir los que provienen de lógica sustractiva por separado en el rango 1(I)-1000(M)
    
    romanos = {'IV':4,'IX':9,'XL':40,'XC':90,'CD':400,'CM':900, 'I':1,'V':5,'X':10,'L':50,'C':100,'D':500,'M':1000}
    
    # Inicializamos algunas variables:
    
    i = 0
    valor = 0
    
    # Lanzamos un bucle de lectura, obsérvese el proceso, lee bloques de 2 en dos y determina si ese bloque está en 
    # nuestro diccionario de dígitos, en caso de que lo esté, lo interpreta con el valor indicado por el diccionario
    # y lo añade al valor acumulado previo y continúa con la lectura del string en el valor posterior a ese dígito.
    # Si no lo está, coge únicamente el primero de ellos, lo interpreta, lo añade y continúa desde el siguiente a ese.
    
    # En caso de que un valor de entrada no sea un dígito romano, nos devuelve un mensaje que nos lo indica.
    
    while i < len(num):
        if num[i:i+2] in romanos:
            valor = valor + romanos[num[i:i+2]]
            i = i+2
            
        else:
            if num[i] in romanos:
                valor = valor + romanos[num[i]]
                i = i+1
            else:
                return("El valor a interpretar no es un número romano")
            
    return(valor)

A continuación viene la parte más delicada de explicar del proceso de elaboración de columnas. Tenemos nombres con el siguiente formato: "077-080  XIV.Enfermedades del sistema genitourinario".

Durante la función de importación que elaboramos tras esta siguiente función a definir y que es la función principal que contiene estas subfunciones que estamos definiendo vamos a hacer en un determinado momento una operación con estos strings que los divide utilizando como referencia el caracter ".".

La parte de la derecha la guardará para que figure en una columna de nombres de grupos, que figurará en el DataFrame final. La parte de la izquierda, que en nuestro caso sería de la forma: "077-080  XIV", la asignará a la columna código numérico, que también figura en el DataFrame final. Para elaborar las columnas de numeral romano y numeral árabe, necesitamos coger sólo la parte en numerales romanos de esta columna "código numérico", y esto exactamente es lo que hace la función que introducimos y explicamos a continuación.

Como en el ejemplo se nos pide que los numerales romanos que encontremos como VI-VIII los traduzcamos sólo a partir del primero de ellos (VI = 6), se selecciona únicamente la primera ocurrencia de este tipo de patrón encontrada en el string a analizar por la función.

In [3]:
def romano(string):
    
    """
    Función que toma un string con números y letras en mayúsculas y extrae de él la primera
    secuencia de caracteres consecutivos en mayúsculas que encuentra.
        
    Parameters
    ----------
    string : string
        String que representa un código numérico con números árabes seguidos de numerales romanos.
    
    Returns
    -------
    palabra: string
        Primera coincidencia de string que compongan un numeral romano en la secuencia de entrada.
    None
        En caso de no encontrar un numeral romano (Secuencia de letras en mayúsculas).
    
    
    Example
    -------
    >>> romano("678-123 XXV")
    XXV
    >>> romano("1234 CXXIX")
    CXXIX
    >>> romano("1234 X-XV")
    X
    >>> romano("1234")
    None
    """
    
    # Utilizamos la función search, el primer argumento es una expresión regular que indica que busque secuencias de
    # letras de la A a la Z en mayúsculas, el segundo indica el string en el que buscar. En caso de no encontrar resultados
    # devuelve None, si los encuentra devuelve la secuencia encontrada. (Esta función genera una salida que nos da los
    # índices de la secuencia encontrada y la secuencia encontrada. La manera que he encontrado de guardar la secuencia
    # es con la función group, en este caso con el argumento 0 para que sea el primer grupo que es el único que
    # encuentra ya que search solo guarda la primera ocurrencia.
    # Posiblemente existan métodos más elegantes de hacerlo, pero este me funciona y no parece generar problemas
    # de rendimiento.)
    
    if re.search("[A-Z]+", string) == None:
        return(None)
    else:
        palabra = re.search("[A-Z]+", string).group(0)
        return(palabra)

Podemos poner a prueba nuestra función:

In [26]:
lista = ["234-54 CV", "444-555 XXXI", "1234 CXXIX", "1234 X-XV", "1234"]

for i in lista:
    print(i+"--->", romano(i))

234-54 CV---> CV
444-555 XXXI---> XXXI
1234 CXXIX---> CXXIX
1234 X-XV---> X
1234---> None


Vamos a elaborar la función principal de este apartado. Podemos resumir sus funciones de la siguiente manera:
    
   ⦁ El parámetro de entrada será el nombre del csv que deseamos importar.
   
   ⦁ Importar el csv con los datos de manera que los campos inciales vengan bien tipados, es decir, importe los string como tal y haga las transformaciones necesarias de los campos numéricos para que se traten correctamente como enteros.
   
   ⦁ Seleccione solo las filas de este DF que hagan referencia a causas de muerte agrupadas, para lo que utilizaremos a función "es_grupo_y_no_total" definida previamente.
   
   ⦁ A partir del campo "Causa de muerte" haga una división en dos columnas, una con los nombres de las causas que figurará en el DF de salida y otra con los códigos de las causas que también figurará.
   
   ⦁ Seleccione la parte correspondiente al numeral romano del código numérico y lo separe en otra columna, tras esto, traducir los numerales presentes en esta a enteros árabes que figurarán en otra columna. Esas dos columnas están presentes en el DF final.
   
   ⦁ Vamos a eliminar las columnas con información sobre el rango de edades y los sexos para que no aparezcan en el DF de salida ya que no son requeridos.
    
Todos estos pasos serán debidamente detallados con comentarios en el código de definición de la función que se expone a continuación:

In [51]:
import re
import pandas as pd

def cargar_datos_mod(nombre_csv):
    
    """
    Función que toma como argumento el nombre del csv a importar y devuelve un PandasDF con las columnas:
       ("Código", "Num.romano", "Num.árabe", "Nombre", "Periodo", "Total").
       
       - "Código"(str): Código numérico de la causa de muerte con numerales índices enteros más romanos.
       - "Num.romano"(str): Parte correspondiente al numeral romano del "Código".
       - "Num.árabe"(int): Parte correspondiente al numeral árabe del "Código".
       - "Nombre"(str): Nombre de la causa de muerte, sin numerales.
       - "Periodo"(int): Año que referencia el periodo temporal de la magnitud "Total".
       - "Total"(int): Total de fallecimientos en el periodo dado por la causa correspondiente.
       
    Adicionalmente devuelve un parámetro denominado "eliminadas" que se corresponde con el número de filas del DF
    original que no pasaron el filtro de la función "es_grupo_y_no_total" y por lo tanto no figuran en el DF final.
        
    Parameters
    ----------
    nombre_csv : string
        String que representa el nombre del csv a importar.
    
    Returns
    -------
    (DF_limpio, eliminadas): (string, int)
        DF_limpio: Un PandasDF que contiene las columnas detalladas anteriormente.
        eliminadas: Número de filas que figuraban en el DF original menos las del DF final.
    
    """
    
    # Comenzamos cargando el csv y definiendo los tipos adecuadamente como ya se vio en el apartado a).
    
    
    DF_sucio = pd.read_csv(nombre_csv, delimiter = ";",dtype = {"Total": str})
    
    DF_sucio["Total"] = DF_sucio["Total"].replace(regex=["\."],value="")
    
    DF_sucio = DF_sucio.astype({"Causa de muerte":"string",
                                "Sexo":"string",
                                "Edad":"string",
                                "Periodo":"int32",
                                "Total": "int64"})
    
    # Elaboramos una lista con las causas de muerte a partir de la correspondiente columna para pasarla de argumento
    # a la función que elaborará la máscara "es_grupo_y_no_total" que fue definida en el apartado b2).
    
    lista = DF_sucio["Causa de muerte"]
    
    cond = [(es_grupo_y_no_total(i)) for i in lista]
    
    # Utilizamos esta máscara para quedarnos solo con las líneas deseadas. El proceso es sencillo, ya que la máscara tiene
    # una logitud similar a la de la columna "Causa de muerte", ya que ha sido elaborada pasando cada elemento de
    # esta columna como argumento a la función "es_grupo_y_no_total" secuencialmente. Ya comentamos que este sería el 
    # proceso a seguir y el objetivo final de la función elaborada en el apartado 2b).
    
    DF_limpio = DF_sucio[cond]
    
    # Creamos dos listas vacías listas para llenarlas con el siguiente bucle, contendrán los elementos que van a figurar
    # en las columnas 
    
    Numeral = []
    Strings = []
    
    # Utilizamos la función split para separar el contenido de la primera columna DF_limpio.iloc[i,0] secuencialmente
    # en dos partes que figuran en una lista, la primera es la parte (lista[0]) numérica del código que figurará en 
    # la columna "Código" y la segunda (lista[1]) es la parte del nombre del código que irá a la columna "Nombre".
    # El proceso se hace añadiendo cada uno de los elementos de estos pares a sus respectivas listas, Numeral y Strings
    # definidas previamente y tras estar estas completamente construidas se asignarán a sus respectivas columnas del DF.
    # El split se hace tomando como caracter de separación ".", y como es un caracter reservado tenemos que denotarlo 
    # como "\."
    
    
    for i in range(len(DF_limpio)):
        lista = re.split("\.", DF_limpio.iloc[i,0])
        
        Numeral.append(lista[0])
        Strings.append(lista[1])
        
    DF_limpio["Código"] = Numeral
    DF_limpio["Nombre"] = Strings
    
    # Ahora aplicamos la función "romano" definida y explicada previamente para separar la parte correspondiente al numeral romano
    # de la columna "Código". Guardamos los resultados en una nueva columna "Num.romano".
        
    DF_limpio["Num.romano"] = [romano(i) for i in DF_limpio["Código"]]
    
    # Y traducimos la columna "Num.romano" a valores enteros y los guardamos en la columna "Num.árabe".
    
    DF_limpio["Num.árabe"] = [romano_a_entero(i) for i in DF_limpio["Num.romano"]]
    
    # Hacemos el drop (eliminamos) las columnas que no son necesarias del DF original. La función drop en nuestro caso
    # toma como argumento el nombre de la columna y el eje en el que operar.
    
    DF_limpio.drop("Sexo", axis=1)
    DF_limpio.drop("Edad", axis=1)
    
    # Hacemos drop también de esta columna antigua que ya hemos descompuesto.
    
    DF_limpio.drop("Causa de muerte", axis=1)
    
    # Reordenamos las columnas que siguen presentes para que aparezcan en el orden deseado.
    
    DF_limpio = DF_limpio.reindex(columns = ["Código", "Num.romano", "Num.árabe", "Nombre", "Periodo", "Total"])
    
    # E imponemos los tipos para asegurarnos de que el DF tiene el formato y forma que deseamos.
    
    DF_limpio = DF_limpio.astype({"Código":"string",
                                  "Num.romano":"string",
                                  "Num.árabe":"int32",
                                  "Nombre":"string",
                                  "Periodo":"int32",
                                  "Total":"int64"})
    
    # Computamos el total de filas eliminadas por el proceso de filtrado con "es_grupo_y_no_total"
        
    eliminadas = (len(DF_sucio)-len(DF_limpio))

    return(DF_limpio, eliminadas)

Ha llegado el momento de probar nuestra función de importación. He incluido una función para eliminar warnings en el siguiente chunk ya que considera peligrosas hacer operaciones del tipo:

    - DF_limpio["Código"] = Numeral
    - DF_limpio["Num.romano"] = [romano(i) for i in DF_limpio["Código"]]
    
En las que estamos copiando el contenido de una lista directamente en una columna (aunque la hemos creado expresamente para ello). De cualquier manera, el resultado es bueno, como podremos ver en las comprobaciones, y la alternativa que se presenta a esta operación, que el propio warning nos aconseja es extremadamente lenta trabajando con DataFrames e incurre en una tremenda penalización al rendimiento. A continuación muestro cómo sería la operación de elaboración de las columnas con la solución sugerida:

    for i in range(len(DF_limpio)):
        
        lista = re.split("\.", DF_limpio.iloc[i,0])
        
        DF_limpio.loc[i, "Código"] = lista[0]
        DF_limpio.loc[i, "Nombre"] = lista[1]

Como ya he comentado, esa operación, aunque es la sugerida y daría el mismo resultado, es extremedamente costosa en términos de rendimiento y por lo tanto es descartada.

In [52]:
import warnings
warnings.filterwarnings('ignore')

datos, num_lin_descartadas = cargar_datos_mod("ine_mortalidad_espanna.csv")

Si se quiere consultar cuales son los fallos que nos indicaba se puede ejecutar el siguiente chunk. Los datos se crean de la misma manera pero expone los warnings.

In [53]:
datos, num_lin_descartadas = cargar_datos_mod("ine_mortalidad_espanna.csv")

Tras ejecutar la función, podemos consultar nuestro DF:

In [54]:
datos

,Código,Num.romano,Num.árabe,Nombre,Periodo,Total
2574,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2018,6398
2575,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2017,6819
2576,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2016,7033
2577,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2015,7567
2578,001-008 I,I,1,Enfermedades infecciosas y parasitarias,2014,6508
...,...,...,...,...,...,...
267691,090-102 XX,XX,20,Causas externas de mortalidad,1984,52
267692,090-102 XX,XX,20,Causas externas de mortalidad,1983,60
267693,090-102 XX,XX,20,Causas externas de mortalidad,1982,61
267694,090-102 XX,XX,20,Causas externas de mortalidad,1981,63


Podemos consultar el número de líneas que han quedado y las que han sido descartadas con respecto al DF original:

In [55]:
print("Número de líneas eliminadas:", num_lin_descartadas)
print("Longitud del DF final:", len(datos))

Número de líneas eliminadas: 265122
Longitud del DF final: 36036


Podemos consultar algunas de las filas:

In [56]:
datos.iloc[[34, 1001, 13000, 20000, 25000]]

,Código,Num.romano,Num.árabe,Nombre,Periodo,Total
2608,001-008 I,I,1,Enfermedades infecciosas y parasitarias,1984,3232
3575,001-008 I,I,1,Enfermedades infecciosas y parasitarias,1992,16
141700,050-052 VI-VIII,VI,6,Enfermedades del sistema nervioso y de los órg...,2005,19
179588,062-067 X,X,10,Enfermedades del sistema respiratorio,1986,31
215476,074-076 XIII,XIII,13,Enfermedades del sistema osteomuscular y del t...,2017,0


Como podemos ver, todas las columnas presentanlos valores esperados, los split se han hecho adecuadamente, las traducciones de numerales también, por lo que podemos concluir que a pesar de los warning todo se ha llevado a cabo adecuadamente.

Por seguir comprobando podemos hacer algo de este estilo, mostrando los diferentes valores para el nombre con sus respectivos valores de las demás columnas asociada para esa fila y así comprobar que, por lo general, todas las traducciones y particiones de datos se han realizado adecuadamente independientemente de la categoría del grupo de enfermedades.

In [63]:
datos.drop_duplicates(["Nombre"])[["Código", "Num.romano", "Num.árabe", "Nombre"]]

,Código,Num.romano,Num.árabe,Nombre
2574,001-008 I,I,1,Enfermedades infecciosas y parasitarias
25740,009-041 II,II,2,Tumores
113256,042-043 III,III,3,Enfermedades de la sangre y de los órganos hem...
120978,044-045 IV,IV,4,"Enfermedades endocrinas, nutricionales y metab..."
128700,046-049 V,V,5,Trastornos mentales y del comportamiento
141570,050-052 VI-VIII,VI,6,Enfermedades del sistema nervioso y de los órg...
151866,053-061 IX,IX,9,Enfermedades del sistema circulatorio
177606,062-067 X,X,10,Enfermedades del sistema respiratorio
195624,068-072 XI,XI,11,Enfermedades del sistema digestivo
213642,074-076 XIII,XIII,13,Enfermedades del sistema osteomuscular y del t...


Una comprobación sencillita de que aparecen todos los años y no se ha colado ningún valores extraño tampoco está de más.

In [59]:
datos.drop_duplicates(["Periodo"])[["Periodo"]]

,Periodo
2574,2018
2575,2017
2576,2016
2577,2015
2578,2014
2579,2013
2580,2012
2581,2011
2582,2010
2583,2009


Por último, aunque los fijamos expresamente durante la función y no debería haber ocurrido nada extraño, podemos consultar los tipos de nuestros datos:

In [57]:
datos.dtypes

Código        string
Num.romano    string
Num.árabe      int32
Nombre        string
Periodo        int32
Total          int64
dtype: object